# Setup and Synthetic Data

In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Set random seed for reproducibility
np.random.seed(42)

# Generate simple 2D data: 10 points, 2 classes
class_0 = np.random.randn(5, 2) + [2, 2]  # Class 0 centered at (2, 2)
class_1 = np.random.randn(5, 2) + [-2, -2]  # Class 1 centered at (-2, -2)
X = np.vstack([class_0, class_1])  # Features (10 points, 2D)
y = np.array([0]*5 + [1]*5)  # Labels: 0 for class_0, 1 for class_1

# Visualize with Plotly
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=y.astype(str),
                 labels={'x': 'Feature 1', 'y': 'Feature 2'},
                 title="Synthetic 2D Dataset for SVM")
fig.show()

# Linear SVM from Scratch

In [2]:
# Convert labels: 0 -> -1, 1 -> 1
y_svm = np.where(y == 0, -1, 1)

# Simple Linear SVM with Gradient Descent
class LinearSVM:
    def __init__(self, learning_rate=0.01, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param  # Regularization strength
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)  # Initialize w
        self.bias = 0  # Initialize b

        # Gradient descent
        for _ in range(self.n_iters):
            for i, x_i in enumerate(X):
                condition = y[i] * (np.dot(x_i, self.weights) + self.bias) >= 1
                if condition:
                    # Update only regularization term
                    self.weights -= self.lr * (2 * self.lambda_param * self.weights)
                else:
                    # Update weights and bias with hinge loss gradient
                    self.weights -= self.lr * (2 * self.lambda_param * self.weights - np.dot(x_i, y[i]))
                    self.bias -= self.lr * (-y[i])

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return np.sign(linear_output)  # Returns -1 or 1

# Train the SVM
svm = LinearSVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
svm.fit(X, y_svm)

# Predict and check
predictions = svm.predict(X)
print("Predictions:", predictions)
print("True Labels:", y_svm)

Predictions: [-1. -1. -1. -1. -1.  1.  1.  1.  1.  1.]
True Labels: [-1 -1 -1 -1 -1  1  1  1  1  1]


# Visualize the Decision Boundary

In [3]:
# Generate grid for decision boundary
x_range = np.linspace(min(X[:, 0]), max(X[:, 0]), 50)
y_range = np.linspace(min(X[:, 1]), max(X[:, 1]), 50)
xx, yy = np.meshgrid(x_range, y_range)
grid = np.c_[xx.ravel(), yy.ravel()]

# Compute decision function
Z = np.dot(grid, svm.weights) + svm.bias
Z = Z.reshape(xx.shape)

# Plot with Plotly
fig = go.Figure()

# Add scatter points
fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers',
                         marker=dict(color=y_svm, colorscale='RdBu', size=10),
                         name='Data Points'))

# Add decision boundary (where w^T * x + b = 0)
fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, contours=dict(start=0, end=0, size=1),
                         line=dict(color='black', width=2), name='Decision Boundary'))

# Add margins (where w^T * x + b = ±1)
fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, contours=dict(start=-1, end=1, size=2),
                         line=dict(color='gray', dash='dash'), name='Margins'))

fig.update_layout(title="Linear SVM Decision Boundary", xaxis_title="Feature 1", yaxis_title="Feature 2")
fig.show()